# Django ORM - Praca z danymi (komendy DML i DQL)

### Co potrzebujemy do uruchomienia tego notebooka?
* jupyter
* django-extensions

<code>$ pip install jupyter django-extensions</code>

Więcej informacji znajdziesz [tutaj](https://medium.com/ayuth/how-to-use-django-in-jupyter-notebook-561ea2401852)

## Definicja modelu

<code>class Task(models.Model):
    name = models.CharField(max_length=64)
</code>
<code>
    def __str__(self):
        return f"{self.name}"
</code>

W dalszej części notebooka przyjęto założenie, że powyższy model został stworzony w aplikacji taskapp w pliku models.py oraz, że przygotowowano i zrealizowano migracje dla tego modelu.

In [ ]:
import os
import django
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'rest.settings')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()

In [ ]:
from form_app5.models import Task

## C - CREATE (CRUD) - DML (Data Manipulation Language)

### Klauzula INSERT

In [ ]:
# Metoda I - metoda create menadżera modelu (objects)

res = Task.objects.create(name="Smażenie")

In [ ]:
# Metoda II - metoda save instancji modelu

task = Task()
task.name = "Pieczenie"
task.save()

In [ ]:
# W drugiej metodzie wartości parametrów można oczywiście przekazać w inicjalizatorze.

task = Task(name="Gotowanie")
task.save()

In [ ]:
# Przed przejściem do litery R dodajmy jeszcze kilka wpisów do tabeli, żeby mieć 
# co analizować podczas poznawania metod implementujących instrukcje z rodziny READ.

Task.objects.create(name="Szukanie")
Task.objects.create(name="Szukanie")
Task.objects.create(name="Programowanie")
Task.objects.create(name="Pływanie")
Task.objects.create(name="Pranie")
Task.objects.create(name="Dodawanie")

## R - Read (CRUD) - DQL (Data Query Language)

### Klauzula SELECT

Operacja READ w SQL to instrukcja SELECT z całą swoją rozbudowaną składnią i operatorami takimi jak: LIKE, GROUP_BY, ORDER_BY, HAVING, IN, JOIN, UNION, ...
Menadżer modelu (**.objects**) posiada odpowiednie metody implementujące część tych instrukcji.

In [ ]:
print(dir(Task.objects))

Metoda all menadżera modelu odpowiada instrukcji SELECT *

In [ ]:
tasks = Task.objects.all()

print(tasks)

Metoda all zwraca obiekt klasy QuerySet. Jest to klasa reprezentująca zapytanie do bazy. Obiekt ten istnieje przed wykonaniem zapytania, a po wykonaniu zapytania jest uzupełniana o odpowiedź z bazy. Wśród swoich pól klasa QuerySet posiada atrybut query przechowujący zapytanie sql, które zostanie (lub już zostało) wykonane na bazie.

In [ ]:
print(tasks.query)

Wśród wielu metod udostępnianych przez QuerySet można znaleźć takie, które widzieliśmy już wśród metod udostępnianych przez menadżera modelu, np. all, filter, exclude, union, get, first, last, ...

In [ ]:
print(dir(tasks))

Wynika to z faktu, że każdy menadżer modelu posiada swój własny, początkowy QuerySet. Kiedy wywołujemy na menadżerze modelu metodę, która znajduje się w interfejsie klasy QuerySet (np. all), to menadżer modelu po prostu wywołuje tą metodę na swoim własnym QuerySet-cie (oddelegowuje wykonanie metody do swojego QuerySet-a).

Czyli metoda all działa na obiekcie QuerySet (początkowym QuerySet-cie menadżera modelu) i zwraca obiekt klasy QuerySet. Skoro to co zwraca metoda all to obiekt klasy QuerySet, to ten obiekt posiada takie metody jak all, filter, ... Wynika z tego, że metodę all możemy łańcuchować, tzn. wywoływać jedną po drugiej.

In [ ]:
tasks = Task.objects.all().all().all()
print(tasks)

QuerySet posiada więcej metod, które w wyniku działania zwracają obiekty klasy QuerySet (np. filter, exclude, order_by, ...) i o ile łańcuchownie samej metody all nie ma za bardzo sensu, tak załańcuchowanie np. metody order_by po wywołaniu metody all w celu posortowania wpisów już ma.

In [ ]:
tasks = Task.objects.all().order_by('name')
print(tasks.query)

Struktura umożliwiająca łańcuchowanie metod na tyle często pojawia się w programowaniu, że posiada nawet swoją nazwę. Mówimy, że QuerySet implementuje wzorzec fluent interface (płynny interfejs).

Ale nie wszystkie metody QuerySet (i menadżera modelu) zwracają QuerySet. Na przykład metody first i last zwracają odpowiednio pierwszy i ostatni element QuerySeta (czyli instancje modelu). Takie metody nie zwracają obiektu klasy QuerySet (nie implementują wzorca fluent interface) i dlatego po ich użyciu nie można już użyć żadnej innej metody obiektu QuerySet do łańcuchowania.

In [ ]:
task = Task.objects.first()
print(task)
print(type(task))

In [ ]:
task = Task.objects.last()
print(task)

### Dostęp do wartości w poszczególnych kolumnach wpisu

Do wartości w poszczególnych kolumnach wpisu dostajemy się poprzez notacją obiektową (odwołujemy się do atrybutu instancji modelu). Jaką wartość w kolumnie name ma ostatni wpis z tabelki Task?

In [ ]:
print(task.name)

### Filtry - metody filter i get (klauzula WHERE)

#### Metoda I - filter

Metoda filter menadżera modelu (i Queryset-a) odpowiada klauzuli WHERE.

In [ ]:
task = Task.objects.filter(name="Szukanie")
print(task)

Widzimy, że metoda filter zwraca obiekt QuerySet. QuerySet może być pusty.

In [ ]:
task = Task.objects.filter(name="Coś czego nie ma w tabeli")
print(task)

QuerySet może być jednoelementowy.

In [ ]:
task = Task.objects.filter(name="Programowanie")
print(task)

#### Metoda II - get

W odróżnieniu od metody filter, metoda get zwraca instancję modelu (a nie obiekt klasy QuerySet).

In [ ]:
task = Task.objects.get(name="Programowanie")
print(task)
print(type(task))

Metoda get oczekuje, że w wyniku otrzyma **jeden i tylko jeden** wpis.

Jeżeli zapytanie nie zwróci żadnego wpisu metoda get rzuci wyjątek *DoesNotExist*.

In [ ]:
task = Task.objects.get(name="Coś czego nie ma w tabeli")

Jeżeli zapytanie zwróci więcej niż jeden wpis metoda get rzuci wyjątek *MultipleObjectsReturned*.

In [ ]:
Task.objects.get(name="Szukanie")

Podsumowując, metoda get w odróżnieniu od metody filter:
* zwraca instancję modelu (a nie obiekty klasy QuerySet)
* jeżeli w wyniku filtrowania otrzymamy pustą odpowiedź rzuci wyjątek *DoesNotExist* (a nie zwróci pusty QuerySet)
* jeżeli w wyniku filtrowania otrzymamy więcej niż jeden wpis rzuci wyjątek *MultipleObjectsReturned* (a nie zwróci wieloelementowy QuerySet)

#### Field lookups (operatory klauzuli WHERE)

Klauzula WHERE posiada wiele operatorów takich jak: LIKE, IN, >, <, ... We frameworku Django implementujemy te operatory za pomocą tak zwanych field lookups. Składniowo używanie lookupów polega na dodaniu po nazwie kolumny w filtrze dwóch znaków __ a następnie odpowiedniego słówka, np. contains, startswith, lte (less than or equal), gt (greater than) ...

Znajdźmy wszystkie wpisy z tabeli Task, dla których wartość w kolumnie name rozpoczyna się na Pr.

In [ ]:
tasks = Task.objects.filter(name__startswith="Pr")
print(tasks)

Popatrzmy na sql

In [ ]:
print(tasks.query)

Znajdźmy wszystkie wpisy z tabli Task, dla których wartość w kolumnie id jest większa bądź równa 5.

In [ ]:
tasks = Task.objects.filter(id__gte=5)
print(tasks)

In [ ]:
# sql?
print(tasks.query)

Znajdźmy wszystkie wpisy z tabeli Task, dla których wartość w kolumnie name zawiera ow.

In [ ]:
tasks = Task.objects.filter(name__contains="ow")
print(tasks)

### Indeksowanie, wycinki (operatory LIMIT i OFFSET)

Klasa QuerySet wspiera indeksowanie oraz wycinki

Znajdźmy czwarty wpis w tabeli Task (indeksowanie od 0).

In [ ]:
task = Task.objects.all()[3]
print(task)
print(type(task))  # instancja modelu

Znajdźmy pięc pierwszych wpisów w tabeli Task.

In [ ]:
tasks = Task.objects.all()[:5]
print(tasks)

In [ ]:
# sql?
print(tasks.query)

Znajdźmy wszystkie wpisy w tabeli Task poza pięcioma pierwszymi.

In [ ]:
tasks = Task.objects.all()[5:]
print(tasks)

In [ ]:
# sql?
print(tasks.query)

Znajdźmy co drugi wpis z tabeli Task (wycinki)

In [ ]:
tasks = Task.objects.all()[::2]
print(tasks)
print(type(tasks))

**Uwaga!**

W ostatnim przykładzie wycinek zwraca **listę instancji modelu**, a nie QuerySet.

**Uwaga!** 

QuerySet w odróżnieniu od listy Pythonowej nie obsługuje negatywnych indeksów.